## Ejemplo de Bloom Filters

Créditos

Curso *[CS246](https://web.stanford.edu/class/cs246/): Mining Massive Data Sets*, Prof. J. Leskovec

Se muestra el uso de un Filtro de Bloom sencillo y se provee una breve comparación de desempeño respecto de soluciones alternativas.



### Requisitos

1. Implementación de [Bloom Filter](https://github.com/hiway/python-bloom-filter) o su alternativa [Bloom Filter 2](https://pypi.org/project/bloom-filter2/)  
1. [Natural Language Toolkit](https://www.nltk.org/index.html)

```
$ pip install bloom_filter   
$ pip install nltk
```

__Data sets__

Importar una lista de palabras de nltk y un data set sobre *movie reviews*

In [1]:
import nltk
nltk.download('words')
from nltk.corpus import words

from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

[nltk_data] Downloading package words to /home/ecciadm/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/ecciadm/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
word_list = words.words()
print(f'Dictionary length: {len(word_list)}')
print(word_list[:20])

Dictionary length: 236736
['wing', 'winter', 'wire', 'wise', 'with', 'woman', 'wood', 'wool', 'word', 'work', 'worm', 'wound', 'writing', 'wrong', 'year', 'yellow', 'yes', 'yesterday', 'you', 'young']


Separar la lista de palabras según su connotación positiva o negativa

In [4]:
neg_reviews = []
pos_reviews = []

for fileid in movie_reviews.fileids('neg'):
  neg_reviews.extend(movie_reviews.words(fileid))
for fileid in movie_reviews.fileids('pos'):
  pos_reviews.extend(movie_reviews.words(fileid))

__Task__

Crear un revisor ortográfico de juguete para poner en práctica el uso del filtro bloom.


In [5]:
from bloom_filter import BloomFilter

word_filter = BloomFilter(max_elements=236736)

for word in word_list:
  word_filter.add(word)

word_set = set(word_list)

Ahora tenemos 3 variables con datos

1.```word_list```, una lista en Python con el diccionario de palabras que descargamos (*case sensitive*)  
2.```word_filter```, un Filtro de Bloom que ya tiene todas las palabras del diccionario  
3.```word_set```, un [Python set](https://docs.python.org/3.6/library/stdtypes.html#set-types-set-frozenset) que también tiene las palabras del diccionario  

Podemos ver el tamaño de cada estructura de datos mediante [getsizeof()](https://docs.python.org/3/library/sys.html#sys.getsizeof)

In [6]:
from sys import getsizeof

swlist = getsizeof(word_list)
swfilter = getsizeof(word_filter)
swset = getsizeof(word_set)

print(f'Size of word_list (in bytes): {swlist}')
print(f'Size of word_filter (in bytes): {swfilter}')
print(f'Size of word_set (in bytes): {swset}')

Size of word_list (in bytes): 2115944
Size of word_filter (in bytes): 48
Size of word_set (in bytes): 8388824


Se puede notar lo eficiente que es el Filtro Bloom en uso de memoria.

In [7]:
print(f'{swlist/swfilter:.2f} Más pequeño que la lista')
print(f'{swset/swfilter:.2f} Más pequeño que el set')

44082.17 Más pequeño que la lista
174767.17 Más pequeño que el set


El Filtro de Bloom está hecho para realizar una operación especial que es probar la membresía a un conjunto. 

Para ver qué tan rápido lo hace usamos el comando `%timeit` de `IPython` que muestra el tiempo de ejecución de forma repetida de una instrucción individual.


In [16]:
%timeit -n 3 "California" in word_list
%timeit -n 3 "California" in word_filter
%timeit -n 3 "California" in word_set

The slowest run took 12.09 times longer than the fastest. This could mean that an intermediate result is being cached.
2.33 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
16.9 µs ± 3.7 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)
141 ns ± 120 ns per loop (mean ± std. dev. of 7 runs, 3 loops each)


1. `word_list` hace una búsqueda lineal  
1. `word_filter` hace múltiples cálculos de funciones hash  
1. `word_set` hace una sola función hash


Con las estructuras de datos que se tienen podemos crear la base del revisor ortográfico.  

La siguiente función recibe dos argumentos, el primero es una lista de palabras y la segunda es una de las estructuras de datos que tenemos. La función retorna la cantidad de palabras que no aparecen en el diccionario o estructura dada.


In [17]:
def not_in_the_list(words, a_list):
  count = 0
  for word in words:
    if (word not in a_list):
      count +=1
  return count

In [18]:
%timeit -n 3 not_in_the_list(["Aani", "roca", "tititi", "Ababua"], word_list)

12.1 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [19]:
%timeit -r 3 not_in_the_list(["Aani", "roca", "tititi", "Ababua"], word_filter)

30.7 µs ± 1.17 µs per loop (mean ± std. dev. of 3 runs, 10000 loops each)


In [20]:
%timeit  -r 3 not_in_the_list(["Aani", "roca", "tititi", "Ababua"], word_set)

294 ns ± 4.07 ns per loop (mean ± std. dev. of 3 runs, 1000000 loops each)
